In [1]:
import pandas as pd
import numpy as np
import librosa
from librosa import display
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import Adam
from keras.utils import to_categorical

import time

In [2]:
# Here we will convert all the mp3 files to wav files
import subprocess
import os 

# Get a list of file paths
base_path = 'train_6_audio/'
bird_dirs = os.listdir(base_path)
bird_dirs = [base_path + bird_dir + '/' for bird_dir in bird_dirs]
bird_dirs = list(filter(lambda x: '.' not in x, bird_dirs))
bird_paths = [bird_dir + bird_file for bird_dir in bird_dirs for bird_file in os.listdir(bird_dir)]
wav_paths = [bird_dir + bird_file for bird_dir in bird_dirs for bird_file in os.listdir(bird_dir)]
wav_paths = list(filter(lambda x: '.wav' in x, wav_paths))

for wav_path in wav_paths:
    subprocess.run(['rm', wav_path])

def convert_to_wav(path):
    wav_path = path.strip('.mp3')
    wav_path = wav_path + '.wav'
    subprocess.run(['ffmpeg', '-i', path, wav_path])
    return wav_path

def delete_file(path):
    subprocess.run(['rm', path])
    return

#convert_to_wav(bird_paths)

In [3]:
meta_csv_path = 'train.csv'
meta_data = pd.read_csv(meta_csv_path)
# We drop all audio files with a rating below 4 points, this still leaves some 15 thousand files.
meta_data = meta_data[meta_data.rating >= 4]
# We select the six birds we are working with
choosen_species = set(['amecro',
                       'grhowl', 
                       'moudov', 
                       'brdowl', 
                       #'belkin1', 
                       'semplo',
                      ])
meta_data = meta_data[meta_data.ebird_code.isin(choosen_species)]
meta_data.shape

(354, 35)

In [4]:
import noisereduce as nr
import wavefile
import soundfile as sf

def reduce_noise(path):
    file = wavefile.load(path)
    samplerate = file[0]
    data = file[1][0]
    nr_data = nr.reduce_noise(audio_clip=np.array(data), noise_clip=np.array(data[samplerate:2*samplerate]),
                                   verbose=False)
    sf.write(path, np.array(list(np.float_(nr_data))), samplerate)
    return path


/home/apolissky/Documents/UChicago/Fuglakall/Fuglakall/lib/python3.8/site-packages/noisereduce/noisereduce.py:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [5]:
#test_dataset = meta_data.sample(n=10)
#test_dataset.shape
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

def get_fp_from_df(df):
    base_path = "train_audio/"
    codes = df['ebird_code']
    filenames = df['filename']
    path_suffix = list(zip(codes,filenames))
    path_suffix = list(map(lambda x: x[0]+'/'+x[1], path_suffix))
    paths = [base_path + path for path in path_suffix]
    return paths

def get_cluster_averages(df):
    df_t = df.transpose()
    kmeans = KMeans(n_clusters=2)
    y_pred = kmeans.fit_predict(df_t)
    df_t['cluster'] = y_pred.tolist()
    cluster1 = df_t[df_t['cluster'] == 0]
    cluster2 = df_t[df_t['cluster'] == 1]
    cluster1 = cluster1.drop(['cluster'],axis=1)
    cluster2 = cluster2.drop(['cluster'],axis=1)
    c1 = np.mean(cluster1.to_numpy(),axis=0)
    c2 = np.mean(cluster2.to_numpy(),axis=0)
    c1std = np.std(cluster1.to_numpy(),axis=0)
    c2std = np.std(cluster2.to_numpy(),axis=0)
    return np.concatenate((c1,c2,c1std,c2std))

def get_mfccs(path):
    data, sample_rate = librosa.load(path, res_type='kaiser_fast')
    mfcc = librosa.feature.mfcc(y=data, sr=sample_rate, n_mfcc=15)
    mfcc_df = pd.DataFrame(mfcc)
    mfcc_processed = get_cluster_averages(mfcc_df)
    return (mfcc_processed,sample_rate)
    
def extract_features(df):
    mp3_paths = get_fp_from_df(df)
    mfcc_list = []
    librosa_srs = []
    for path in mp3_paths:
        wav_path = convert_to_wav(path)
        reduce_noise(wav_path)
        extracted = get_mfccs(wav_path)
        mfcc_list.append(extracted[0])
        librosa_srs.append(extracted[1])
        delete_file(wav_path)
    df['mfccs'] = mfcc_list
    df['librosa_sample_rate'] = librosa_srs
    return df

start_time = time.time()
feature_df = extract_features(meta_data)
stop_time = time.time()
print("Feature extraction took {} seconds".format(stop_time-start_time))


Feature extraction took 889.4930160045624 seconds


In [6]:
# Convert features and output labels to numpy arrays
X = np.array(feature_df.mfccs.tolist())
y = np.array(feature_df.ebird_code.tolist())

# Label Encoder
le = LabelEncoder()
yy = to_categorical(le.fit_transform(y))

x_train, x_test, y_train, y_test = train_test_split(X, yy, test_size=0.2, random_state=127)

In [7]:
# The actuall feedforward neural network architecture
num_labels = yy.shape[1]
filter_size = 2
def build_model_graph(input_shape=(40,)):
    model = Sequential()
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_labels))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
    return model

model = build_model_graph()
model.layers
#model.summary()
score = model.evaluate(x_test, y_test, verbose=0)
accuracy = 100*score[1]

print("Pre-training accuracy: {}".format(accuracy))

Pre-training accuracy: 40.84506928920746


In [9]:
from keras.callbacks import ModelCheckpoint
#from datetime import datetime

num_epochs = 2500
num_batch_size = 394

start_time = time.time()
model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_test,y_test),verbose=1)
stop_time = time.time()
print("Training time: {}".format(stop_time-start_time))

Epoch 1/2500
1/1 [==============================] - 0s 39ms/step - loss: 0.0338 - accuracy: 0.9859 - val_loss: 1.6525 - val_accuracy: 0.7887
Epoch 2/2500
1/1 [==============================] - 0s 21ms/step - loss: 0.0204 - accuracy: 0.9929 - val_loss: 1.6195 - val_accuracy: 0.7887
Epoch 3/2500
1/1 [==============================] - 0s 27ms/step - loss: 0.0193 - accuracy: 0.9965 - val_loss: 1.5920 - val_accuracy: 0.7887
Epoch 4/2500
1/1 [==============================] - 0s 26ms/step - loss: 0.0172 - accuracy: 0.9929 - val_loss: 1.5674 - val_accuracy: 0.7887
Epoch 5/2500
1/1 [==============================] - 0s 19ms/step - loss: 0.0432 - accuracy: 0.9894 - val_loss: 1.5399 - val_accuracy: 0.7887
Epoch 6/2500
1/1 [==============================] - 0s 22ms/step - loss: 0.0191 - accuracy: 0.9965 - val_loss: 1.5189 - val_accuracy: 0.7746
Epoch 7/2500
1/1 [==============================] - 0s 23ms/step - loss: 0.0231 - accuracy: 0.9929 - val_loss: 1.5134 - val_accuracy: 0.7887
Epoch 8/2500


Epoch 59/2500
1/1 [==============================] - 0s 26ms/step - loss: 0.0293 - accuracy: 0.9965 - val_loss: 1.5570 - val_accuracy: 0.7887
Epoch 60/2500
1/1 [==============================] - 0s 25ms/step - loss: 0.0348 - accuracy: 0.9859 - val_loss: 1.5494 - val_accuracy: 0.7887
Epoch 61/2500
1/1 [==============================] - 0s 22ms/step - loss: 0.0372 - accuracy: 0.9859 - val_loss: 1.5382 - val_accuracy: 0.7887
Epoch 62/2500
1/1 [==============================] - 0s 22ms/step - loss: 0.0159 - accuracy: 0.9965 - val_loss: 1.5310 - val_accuracy: 0.7887
Epoch 63/2500
1/1 [==============================] - 0s 23ms/step - loss: 0.0179 - accuracy: 0.9929 - val_loss: 1.5177 - val_accuracy: 0.7887
Epoch 64/2500
1/1 [==============================] - 0s 21ms/step - loss: 0.0258 - accuracy: 0.9894 - val_loss: 1.5075 - val_accuracy: 0.7887
Epoch 65/2500
1/1 [==============================] - 0s 20ms/step - loss: 0.0181 - accuracy: 0.9965 - val_loss: 1.5027 - val_accuracy: 0.7887
Epoch 

1/1 [==============================] - 0s 18ms/step - loss: 0.0348 - accuracy: 0.9859 - val_loss: 1.5817 - val_accuracy: 0.7746
Epoch 117/2500
1/1 [==============================] - 0s 23ms/step - loss: 0.0217 - accuracy: 0.9929 - val_loss: 1.5707 - val_accuracy: 0.7606
Epoch 118/2500
1/1 [==============================] - 0s 21ms/step - loss: 0.0412 - accuracy: 0.9894 - val_loss: 1.5686 - val_accuracy: 0.7606
Epoch 119/2500
1/1 [==============================] - 0s 21ms/step - loss: 0.0316 - accuracy: 0.9788 - val_loss: 1.5765 - val_accuracy: 0.7606
Epoch 120/2500
1/1 [==============================] - 0s 21ms/step - loss: 0.0298 - accuracy: 0.9894 - val_loss: 1.5805 - val_accuracy: 0.7606
Epoch 121/2500
1/1 [==============================] - 0s 22ms/step - loss: 0.0359 - accuracy: 0.9894 - val_loss: 1.5929 - val_accuracy: 0.7606
Epoch 122/2500
1/1 [==============================] - 0s 22ms/step - loss: 0.0090 - accuracy: 0.9965 - val_loss: 1.6066 - val_accuracy: 0.7606
Epoch 123/2500

1/1 [==============================] - 0s 23ms/step - loss: 0.0271 - accuracy: 0.9929 - val_loss: 1.6075 - val_accuracy: 0.7746
Epoch 174/2500
1/1 [==============================] - 0s 22ms/step - loss: 0.0202 - accuracy: 0.9965 - val_loss: 1.6020 - val_accuracy: 0.7746
Epoch 175/2500
1/1 [==============================] - 0s 22ms/step - loss: 0.0699 - accuracy: 0.9894 - val_loss: 1.5944 - val_accuracy: 0.7746
Epoch 176/2500
1/1 [==============================] - 0s 23ms/step - loss: 0.0436 - accuracy: 0.9859 - val_loss: 1.5925 - val_accuracy: 0.7606
Epoch 177/2500
1/1 [==============================] - 0s 18ms/step - loss: 0.0541 - accuracy: 0.9859 - val_loss: 1.5886 - val_accuracy: 0.7606
Epoch 178/2500
1/1 [==============================] - 0s 18ms/step - loss: 0.0180 - accuracy: 0.9929 - val_loss: 1.5862 - val_accuracy: 0.7606
Epoch 179/2500
1/1 [==============================] - 0s 18ms/step - loss: 0.0249 - accuracy: 0.9929 - val_loss: 1.5839 - val_accuracy: 0.7606
Epoch 180/2500

1/1 [==============================] - 0s 22ms/step - loss: 0.0580 - accuracy: 0.9823 - val_loss: 1.7323 - val_accuracy: 0.7746
Epoch 231/2500
1/1 [==============================] - 0s 19ms/step - loss: 0.0309 - accuracy: 0.9894 - val_loss: 1.7481 - val_accuracy: 0.7746
Epoch 232/2500
1/1 [==============================] - 0s 19ms/step - loss: 0.0132 - accuracy: 0.9965 - val_loss: 1.7591 - val_accuracy: 0.7746
Epoch 233/2500
1/1 [==============================] - 0s 24ms/step - loss: 0.0317 - accuracy: 0.9859 - val_loss: 1.7643 - val_accuracy: 0.7746
Epoch 234/2500
1/1 [==============================] - 0s 23ms/step - loss: 0.0229 - accuracy: 0.9929 - val_loss: 1.7717 - val_accuracy: 0.7887
Epoch 235/2500
1/1 [==============================] - 0s 18ms/step - loss: 0.0731 - accuracy: 0.9823 - val_loss: 1.7633 - val_accuracy: 0.7887
Epoch 236/2500
1/1 [==============================] - 0s 19ms/step - loss: 0.0235 - accuracy: 0.9859 - val_loss: 1.7585 - val_accuracy: 0.7746
Epoch 237/2500

...

Epoch 2225/2500
1/1 [==============================] - 0s 18ms/step - loss: 0.0022 - accuracy: 1.0000 - val_loss: 2.1787 - val_accuracy: 0.7887
Epoch 2226/2500
1/1 [==============================] - 0s 23ms/step - loss: 0.0034 - accuracy: 1.0000 - val_loss: 2.2301 - val_accuracy: 0.7887
Epoch 2227/2500
1/1 [==============================] - 0s 24ms/step - loss: 0.0418 - accuracy: 0.9965 - val_loss: 2.2544 - val_accuracy: 0.7887
Epoch 2228/2500
1/1 [==============================] - 0s 23ms/step - loss: 0.0453 - accuracy: 0.9788 - val_loss: 2.2800 - val_accuracy: 0.7887
Epoch 2229/2500
1/1 [==============================] - 0s 19ms/step - loss: 0.0251 - accuracy: 0.9894 - val_loss: 2.2954 - val_accuracy: 0.7887
Epoch 2230/2500
1/1 [==============================] - 0s 19ms/step - loss: 0.0429 - accuracy: 0.9788 - val_loss: 2.3148 - val_accuracy: 0.7887
Epoch 2231/2500
1/1 [==============================] - 0s 22ms/step - loss: 0.0315 - accuracy: 0.9894 - val_loss: 2.3284 - val_accuracy:

Epoch 2282/2500
1/1 [==============================] - 0s 18ms/step - loss: 0.0477 - accuracy: 0.9929 - val_loss: 2.2151 - val_accuracy: 0.7746
Epoch 2283/2500
1/1 [==============================] - 0s 24ms/step - loss: 0.0039 - accuracy: 1.0000 - val_loss: 2.2056 - val_accuracy: 0.7606
Epoch 2284/2500
1/1 [==============================] - 0s 22ms/step - loss: 0.0149 - accuracy: 0.9929 - val_loss: 2.2027 - val_accuracy: 0.7606
Epoch 2285/2500
1/1 [==============================] - 0s 24ms/step - loss: 0.0045 - accuracy: 1.0000 - val_loss: 2.2013 - val_accuracy: 0.7606
Epoch 2286/2500
1/1 [==============================] - 0s 21ms/step - loss: 0.0056 - accuracy: 1.0000 - val_loss: 2.2042 - val_accuracy: 0.7606
Epoch 2287/2500
1/1 [==============================] - 0s 22ms/step - loss: 0.0045 - accuracy: 1.0000 - val_loss: 2.2104 - val_accuracy: 0.7606
Epoch 2288/2500
1/1 [==============================] - 0s 24ms/step - loss: 0.0155 - accuracy: 0.9965 - val_loss: 2.2057 - val_accuracy:

Epoch 2339/2500
1/1 [==============================] - 0s 24ms/step - loss: 0.0028 - accuracy: 1.0000 - val_loss: 2.2115 - val_accuracy: 0.7887
Epoch 2340/2500
1/1 [==============================] - 0s 21ms/step - loss: 0.0075 - accuracy: 0.9965 - val_loss: 2.2040 - val_accuracy: 0.7887
Epoch 2341/2500
1/1 [==============================] - 0s 18ms/step - loss: 0.0308 - accuracy: 0.9823 - val_loss: 2.2012 - val_accuracy: 0.7887
Epoch 2342/2500
1/1 [==============================] - 0s 22ms/step - loss: 0.0032 - accuracy: 1.0000 - val_loss: 2.2066 - val_accuracy: 0.7887
Epoch 2343/2500
1/1 [==============================] - 0s 23ms/step - loss: 0.0356 - accuracy: 0.9929 - val_loss: 2.2115 - val_accuracy: 0.7887
Epoch 2344/2500
1/1 [==============================] - 0s 17ms/step - loss: 0.0182 - accuracy: 0.9894 - val_loss: 2.2123 - val_accuracy: 0.7887
Epoch 2345/2500
1/1 [==============================] - 0s 21ms/step - loss: 0.0147 - accuracy: 0.9965 - val_loss: 2.2179 - val_accuracy:

Epoch 2396/2500
1/1 [==============================] - 0s 21ms/step - loss: 0.0164 - accuracy: 0.9929 - val_loss: 2.3998 - val_accuracy: 0.7887
Epoch 2397/2500
1/1 [==============================] - 0s 18ms/step - loss: 0.0085 - accuracy: 0.9965 - val_loss: 2.3959 - val_accuracy: 0.7887
Epoch 2398/2500
1/1 [==============================] - 0s 19ms/step - loss: 0.0231 - accuracy: 0.9894 - val_loss: 2.3491 - val_accuracy: 0.7887
Epoch 2399/2500
1/1 [==============================] - 0s 18ms/step - loss: 0.0372 - accuracy: 0.9859 - val_loss: 2.3115 - val_accuracy: 0.7887
Epoch 2400/2500
1/1 [==============================] - 0s 18ms/step - loss: 0.0201 - accuracy: 0.9929 - val_loss: 2.2921 - val_accuracy: 0.7887
Epoch 2401/2500
1/1 [==============================] - 0s 21ms/step - loss: 0.0610 - accuracy: 0.9965 - val_loss: 2.2786 - val_accuracy: 0.8028
Epoch 2402/2500
1/1 [==============================] - 0s 18ms/step - loss: 0.0063 - accuracy: 1.0000 - val_loss: 2.2600 - val_accuracy:

Epoch 2453/2500
1/1 [==============================] - 0s 22ms/step - loss: 0.0091 - accuracy: 0.9965 - val_loss: 2.2725 - val_accuracy: 0.7606
Epoch 2454/2500
1/1 [==============================] - 0s 18ms/step - loss: 0.0254 - accuracy: 0.9965 - val_loss: 2.2611 - val_accuracy: 0.7746
Epoch 2455/2500
1/1 [==============================] - 0s 21ms/step - loss: 0.0876 - accuracy: 0.9788 - val_loss: 2.2801 - val_accuracy: 0.7746
Epoch 2456/2500
1/1 [==============================] - 0s 18ms/step - loss: 0.0204 - accuracy: 0.9929 - val_loss: 2.2908 - val_accuracy: 0.7606
Epoch 2457/2500
1/1 [==============================] - 0s 25ms/step - loss: 0.0074 - accuracy: 0.9965 - val_loss: 2.2935 - val_accuracy: 0.7465
Epoch 2458/2500
1/1 [==============================] - 0s 23ms/step - loss: 0.0049 - accuracy: 1.0000 - val_loss: 2.3001 - val_accuracy: 0.7465
Epoch 2459/2500
1/1 [==============================] - 0s 23ms/step - loss: 0.0222 - accuracy: 0.9894 - val_loss: 2.2939 - val_accuracy:

In [10]:
score = model.evaluate(x_train, y_train, verbose=0)
print("Training Accuracy: {0:.2%}".format(score[1]))

score = model.evaluate(x_test, y_test, verbose=0)
print("Testing Accuracy: {0:.2%}".format(score[1]))

Training Accuracy: 100.00%
Testing Accuracy: 78.87%


In [12]:
model.save('fuglakall.h5')